# PennyLane Fundamentals for Quantum Machine Learning

This notebook covers quantum machine learning concepts using PennyLane, focusing on differentiable quantum computing.

In [ ]:
import pennylane as qml
import numpy as np
import matplotlib.pyplot as plt
from typing import List, Tuple, Optional, Callable
import torch
import torch.nn as nn
import torch.optim as optim

## 1. Quantum Devices and Qubits

In [ ]:
dev_default = qml.device('default.qubit', wires=4)
print(f"Default device: {dev_default}")

dev_lightning = qml.device('lightning.qubit', wires=4)
print(f"Lightning device: {dev_lightning}")

dev_mixed = qml.device('default.mixed', wires=2)
print(f"Mixed state device: {dev_mixed}")

## 2. Basic Quantum Circuits

In [ ]:
@qml.qnode(dev_default)
def basic_circuit(params):
    qml.RY(params[0], wires=0)
    qml.RX(params[1], wires=1)
    qml.CNOT(wires=[0, 1])
    qml.RZ(params[2], wires=1)
    return qml.expval(qml.PauliZ(0) @ qml.PauliZ(1))

params = np.array([0.5, 1.2, 0.3])
result = basic_circuit(params)
print(f"Circuit output: {result}")

print("\nCircuit structure:")
print(qml.draw(basic_circuit)(params))

## 3. Quantum State Preparation

In [ ]:
@qml.qnode(dev_default)
def state_preparation_circuit():
    qml.Hadamard(wires=0)
    qml.Hadamard(wires=1)
    qml.CNOT(wires=[0, 1])
    return qml.state()

@qml.qnode(dev_default)
def amplitude_encoding(data):
    data = data / np.linalg.norm(data)
    qml.AmplitudeEmbedding(data, wires=range(2), normalize=False)
    return qml.state()

@qml.qnode(dev_default)
def angle_encoding(data):
    for i, angle in enumerate(data):
        qml.RY(angle, wires=i)
    return [qml.expval(qml.PauliZ(i)) for i in range(len(data))]

bell_state = state_preparation_circuit()
print(f"Bell state: {bell_state}")

data_vector = np.array([1, 2, 3, 4])
encoded_state = amplitude_encoding(data_vector)
print(f"\nAmplitude encoded state: {encoded_state}")

angles = np.array([np.pi/4, np.pi/3, np.pi/6, np.pi/2])
angle_result = angle_encoding(angles)
print(f"\nAngle encoding measurements: {angle_result}")

## 4. Variational Quantum Circuits

In [ ]:
def layer(weights):
    for i in range(len(weights[0])):
        qml.RY(weights[0][i], wires=i)
        qml.RZ(weights[1][i], wires=i)
    
    for i in range(len(weights[0]) - 1):
        qml.CNOT(wires=[i, i+1])

@qml.qnode(dev_default)
def variational_circuit(weights, data):
    qml.AngleEmbedding(data, wires=range(len(data)))
    
    for w in weights:
        layer(w)
    
    return [qml.expval(qml.PauliZ(i)) for i in range(len(data))]

n_qubits = 4
n_layers = 3
weights_shape = (n_layers, 2, n_qubits)
weights = np.random.randn(*weights_shape) * 0.1
data = np.random.randn(n_qubits)

output = variational_circuit(weights, data)
print(f"Variational circuit output: {output}")

print("\nCircuit structure:")
print(qml.draw(variational_circuit, expansion_strategy="device")(weights, data))

## 5. Quantum Gradients and Optimization

In [ ]:
@qml.qnode(dev_default)
def cost_circuit(params):
    qml.RX(params[0], wires=0)
    qml.RY(params[1], wires=1)
    qml.CNOT(wires=[0, 1])
    qml.RZ(params[2], wires=0)
    return qml.expval(qml.PauliZ(0) @ qml.PauliX(1))

def cost_function(params):
    return cost_circuit(params)

params = np.array([0.1, 0.2, 0.3], requires_grad=True)

gradient = qml.grad(cost_function)
grad_val = gradient(params)
print(f"Gradient: {grad_val}")

opt = qml.GradientDescentOptimizer(stepsize=0.1)

costs = []
for i in range(50):
    params = opt.step(cost_function, params)
    costs.append(cost_function(params))

print(f"\nOptimized parameters: {params}")
print(f"Final cost: {costs[-1]}")

plt.figure(figsize=(8, 4))
plt.plot(costs)
plt.xlabel('Iteration')
plt.ylabel('Cost')
plt.title('Optimization Progress')
plt.show()

## 6. Quantum Machine Learning Classifier

In [ ]:
def generate_data(n_samples=100):
    X = np.random.randn(n_samples, 2)
    y = (X[:, 0]**2 + X[:, 1]**2 > 1).astype(int)
    return X, y

@qml.qnode(dev_default)
def quantum_classifier(weights, x):
    qml.AngleEmbedding(x, wires=range(2))
    
    for i in range(3):
        qml.RY(weights[i, 0], wires=0)
        qml.RY(weights[i, 1], wires=1)
        qml.CNOT(wires=[0, 1])
        qml.RZ(weights[i, 2], wires=0)
        qml.RZ(weights[i, 3], wires=1)
    
    return qml.expval(qml.PauliZ(0))

def cost_classification(weights, X, y):
    predictions = np.array([quantum_classifier(weights, x) for x in X])
    loss = np.mean((predictions - (2*y - 1))**2)
    return loss

def accuracy(weights, X, y):
    predictions = np.array([quantum_classifier(weights, x) for x in X])
    predicted_labels = (predictions > 0).astype(int)
    return np.mean(predicted_labels == y)

X_train, y_train = generate_data(50)
X_test, y_test = generate_data(20)

weights = np.random.randn(3, 4) * 0.1
opt = qml.AdamOptimizer(stepsize=0.1)

for epoch in range(20):
    weights = opt.step(lambda w: cost_classification(w, X_train, y_train), weights)
    
    if epoch % 5 == 0:
        train_acc = accuracy(weights, X_train, y_train)
        test_acc = accuracy(weights, X_test, y_test)
        print(f"Epoch {epoch}: Train Accuracy = {train_acc:.3f}, Test Accuracy = {test_acc:.3f}")

## 7. Quantum Kernels

In [ ]:
@qml.qnode(dev_default)
def quantum_kernel_circuit(x1, x2):
    n_qubits = len(x1)
    
    qml.AngleEmbedding(x1, wires=range(n_qubits))
    qml.adjoint(qml.AngleEmbedding)(x2, wires=range(n_qubits))
    
    return qml.probs(wires=range(n_qubits))

def quantum_kernel(x1, x2):
    return quantum_kernel_circuit(x1, x2)[0]

def compute_kernel_matrix(X):
    n_samples = len(X)
    K = np.zeros((n_samples, n_samples))
    
    for i in range(n_samples):
        for j in range(i, n_samples):
            K[i, j] = quantum_kernel(X[i], X[j])
            K[j, i] = K[i, j]
    
    return K

X_kernel = np.random.randn(10, 4)
K = compute_kernel_matrix(X_kernel)

plt.figure(figsize=(6, 5))
plt.imshow(K, cmap='viridis')
plt.colorbar()
plt.title('Quantum Kernel Matrix')
plt.xlabel('Sample index')
plt.ylabel('Sample index')
plt.show()

## 8. Quantum Autoencoder

In [ ]:
n_qubits = 4
n_latent = 2
dev_ae = qml.device('default.qubit', wires=n_qubits)

@qml.qnode(dev_ae)
def quantum_autoencoder(encoder_weights, decoder_weights, data):
    qml.AmplitudeEmbedding(data, wires=range(n_qubits), normalize=True)
    
    for i in range(n_qubits):
        qml.RY(encoder_weights[0, i], wires=i)
        qml.RZ(encoder_weights[1, i], wires=i)
    
    for i in range(n_qubits - 1):
        qml.CNOT(wires=[i, i+1])
    
    for i in range(n_latent, n_qubits):
        qml.Hadamard(wires=i)
    
    for i in range(n_qubits - 1):
        qml.CNOT(wires=[i, i+1])
    
    for i in range(n_qubits):
        qml.RY(decoder_weights[0, i], wires=i)
        qml.RZ(decoder_weights[1, i], wires=i)
    
    return qml.state()

def autoencoder_cost(weights, data):
    encoder_weights = weights[:2]
    decoder_weights = weights[2:]
    
    output_state = quantum_autoencoder(encoder_weights, decoder_weights, data)
    
    data_normalized = data / np.linalg.norm(data)
    fidelity = np.abs(np.vdot(data_normalized, output_state[:len(data)]))**2
    
    return 1 - fidelity

test_data = np.random.randn(2**n_qubits)
weights = np.random.randn(4, n_qubits) * 0.1

opt = qml.AdamOptimizer(0.1)
costs_ae = []

for i in range(50):
    weights = opt.step(lambda w: autoencoder_cost(w, test_data), weights)
    cost = autoencoder_cost(weights, test_data)
    costs_ae.append(cost)
    
    if i % 10 == 0:
        print(f"Step {i}: Cost = {cost:.4f}")

plt.figure(figsize=(8, 4))
plt.plot(costs_ae)
plt.xlabel('Training Step')
plt.ylabel('Reconstruction Error')
plt.title('Quantum Autoencoder Training')
plt.show()

## 9. Quantum Natural Gradient Descent

In [ ]:
@qml.qnode(dev_default)
def qng_circuit(params):
    qml.RX(params[0], wires=0)
    qml.RY(params[1], wires=0)
    qml.RZ(params[2], wires=0)
    qml.CNOT(wires=[0, 1])
    qml.RY(params[3], wires=1)
    return qml.expval(qml.PauliZ(0) @ qml.PauliZ(1))

def cost_qng(params):
    return qng_circuit(params)

params_qng = np.random.randn(4) * 0.1

opt_qng = qml.QNGOptimizer(stepsize=0.01)
opt_gd = qml.GradientDescentOptimizer(stepsize=0.01)

costs_qng = []
costs_gd = []
params_qng_track = params_qng.copy()
params_gd_track = params_qng.copy()

for i in range(30):
    params_qng_track = opt_qng.step(cost_qng, params_qng_track)
    params_gd_track = opt_gd.step(cost_qng, params_gd_track)
    
    costs_qng.append(cost_qng(params_qng_track))
    costs_gd.append(cost_qng(params_gd_track))

plt.figure(figsize=(10, 4))
plt.plot(costs_qng, label='Quantum Natural Gradient')
plt.plot(costs_gd, label='Gradient Descent')
plt.xlabel('Iteration')
plt.ylabel('Cost')
plt.title('QNG vs Standard Gradient Descent')
plt.legend()
plt.show()

print(f"Final QNG cost: {costs_qng[-1]:.6f}")
print(f"Final GD cost: {costs_gd[-1]:.6f}")

## 10. Hybrid Quantum-Classical Neural Network

In [ ]:
n_qubits = 4
dev_hybrid = qml.device('default.qubit', wires=n_qubits)

@qml.qnode(dev_hybrid, interface='torch')
def quantum_layer(inputs, weights):
    qml.AngleEmbedding(inputs, wires=range(n_qubits))
    
    qml.BasicEntanglerLayers(weights, wires=range(n_qubits))
    
    return [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)]

class HybridModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.classical1 = nn.Linear(10, n_qubits)
        self.quantum_weights = nn.Parameter(torch.randn(3, n_qubits))
        self.classical2 = nn.Linear(n_qubits, 2)
    
    def forward(self, x):
        x = torch.relu(self.classical1(x))
        x = torch.stack([quantum_layer(x[i], self.quantum_weights) for i in range(x.shape[0])])
        x = self.classical2(x)
        return x

model = HybridModel()
optimizer = optim.Adam(model.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()

X_hybrid = torch.randn(100, 10)
y_hybrid = torch.randint(0, 2, (100,))

losses = []
for epoch in range(10):
    optimizer.zero_grad()
    outputs = model(X_hybrid)
    loss = criterion(outputs, y_hybrid)
    loss.backward()
    optimizer.step()
    losses.append(loss.item())
    
    if epoch % 2 == 0:
        print(f"Epoch {epoch}: Loss = {loss.item():.4f}")

plt.figure(figsize=(8, 4))
plt.plot(losses)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Hybrid Quantum-Classical Network Training')
plt.show()

## 11. Quantum Generative Adversarial Network (QGAN)

In [ ]:
n_qubits_gan = 3
dev_gan = qml.device('default.qubit', wires=n_qubits_gan)

@qml.qnode(dev_gan)
def generator(weights, latent):
    qml.RY(latent[0], wires=0)
    qml.RY(latent[1], wires=1)
    
    for i in range(3):
        for j in range(n_qubits_gan):
            qml.RY(weights[i, j, 0], wires=j)
            qml.RZ(weights[i, j, 1], wires=j)
        
        for j in range(n_qubits_gan - 1):
            qml.CNOT(wires=[j, j+1])
    
    return qml.probs(wires=range(n_qubits_gan))

@qml.qnode(dev_gan)
def discriminator(weights, data):
    qml.AmplitudeEmbedding(data, wires=range(n_qubits_gan), normalize=True)
    
    for i in range(2):
        for j in range(n_qubits_gan):
            qml.RY(weights[i, j, 0], wires=j)
            qml.RZ(weights[i, j, 1], wires=j)
        
        for j in range(n_qubits_gan - 1):
            qml.CZ(wires=[j, j+1])
    
    return qml.expval(qml.PauliZ(0))

def real_data_distribution():
    probs = np.zeros(2**n_qubits_gan)
    probs[0] = 0.5
    probs[-1] = 0.5
    return probs

gen_weights = np.random.randn(3, n_qubits_gan, 2) * 0.1
disc_weights = np.random.randn(2, n_qubits_gan, 2) * 0.1

opt_gen = qml.AdamOptimizer(0.01)
opt_disc = qml.AdamOptimizer(0.01)

for epoch in range(20):
    latent = np.random.randn(2)
    fake_data = generator(gen_weights, latent)
    real_data = real_data_distribution()
    
    def disc_cost(w):
        real_pred = discriminator(w, real_data)
        fake_pred = discriminator(w, fake_data)
        return -(real_pred - fake_pred)
    
    disc_weights = opt_disc.step(disc_cost, disc_weights)
    
    def gen_cost(w):
        latent_new = np.random.randn(2)
        fake_data_new = generator(w, latent_new)
        return -discriminator(disc_weights, fake_data_new)
    
    gen_weights = opt_gen.step(gen_cost, gen_weights)
    
    if epoch % 5 == 0:
        test_latent = np.random.randn(2)
        generated = generator(gen_weights, test_latent)
        print(f"Epoch {epoch}: Generated distribution entropy = {-np.sum(generated * np.log(generated + 1e-10)):.3f}")

## 12. Quantum Transfer Learning

In [ ]:
@qml.qnode(dev_default)
def pretrained_circuit(inputs, weights):
    qml.AngleEmbedding(inputs, wires=range(4))
    
    for i in range(2):
        qml.RY(weights[i, 0], wires=0)
        qml.RY(weights[i, 1], wires=1)
        qml.RY(weights[i, 2], wires=2)
        qml.RY(weights[i, 3], wires=3)
        qml.CNOT(wires=[0, 1])
        qml.CNOT(wires=[2, 3])
        qml.CNOT(wires=[1, 2])
    
    return [qml.expval(qml.PauliZ(i)) for i in range(4)]

pretrained_weights = np.array([
    [0.5, 0.3, -0.2, 0.1],
    [0.2, -0.4, 0.3, 0.5]
])

@qml.qnode(dev_default)
def transfer_learning_circuit(inputs, frozen_weights, trainable_weights):
    qml.AngleEmbedding(inputs, wires=range(4))
    
    for i in range(2):
        for j in range(4):
            qml.RY(frozen_weights[i, j], wires=j)
        qml.CNOT(wires=[0, 1])
        qml.CNOT(wires=[2, 3])
        qml.CNOT(wires=[1, 2])
    
    for j in range(4):
        qml.RY(trainable_weights[j], wires=j)
    
    return qml.expval(qml.PauliZ(0) @ qml.PauliZ(3))

def transfer_cost(trainable_weights, X, y):
    predictions = [transfer_learning_circuit(x, pretrained_weights, trainable_weights) for x in X]
    return np.mean((np.array(predictions) - y)**2)

X_transfer = np.random.randn(20, 4)
y_transfer = np.random.randn(20)

trainable_weights = np.random.randn(4) * 0.1
opt_transfer = qml.AdamOptimizer(0.1)

for epoch in range(10):
    trainable_weights = opt_transfer.step(
        lambda w: transfer_cost(w, X_transfer, y_transfer), 
        trainable_weights
    )
    
    if epoch % 3 == 0:
        cost = transfer_cost(trainable_weights, X_transfer, y_transfer)
        print(f"Epoch {epoch}: Cost = {cost:.4f}")

print(f"\nFinal trainable weights: {trainable_weights}")